In [1]:
from numpy import linalg as LA
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import circle_fit as cf
import numpy as np
import cv2
import timeit
import time
import os

## 目前進度
### 還須找到更好的擬合公式
### 或是找到適合 sample 的距離 

## 縮寫對應
pt   -> points  <br>
dist -> distance <br>
spl  -> sample


## 影像處理

In [2]:
def image_processing(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5,5), 0)
    nlm  = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
    _, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    edge = cv2.Canny(otsu, 0, 20)
    
    return blur, nlm, otsu, edge

def get_edge_points(edge_img):
    edge_points = [[j,i] for i, r in enumerate(edge_img) for j, c in enumerate(r) if c == 255]

    return edge_points

## Get subpixel edge locations

In [3]:
def sigmoid(x, a, b):
    return 1.0 / (1.0 + np.exp(-a*(x-b)))

In [4]:
def get_influential_neighbors(coord):
    ## 依照 sample 的點落在當下 pixel 中的哪個空間，分為 4 個case
    ##     0 . 1
    ##    .......
    ##     2 . 3
    y, x = round(coord[0]), round(coord[1])
    space = np.dot([1,2], [y,x])
    
    switcher = {
        0: [[y,x], [y-1,x], [y-1,x-1], [y,x-1]],
        1: [[y,x], [y,x+1], [y-1,x+1], [y-1,x]],
        2: [[y,x], [y,x-1], [y+1,x-1], [y+1,x]],
        3: [[y,x], [y+1,x], [y+1,x+1], [y,x+1]],
    }
    
    neighbors_loc = np.asarray(switcher.get(space))
    return neighbors_loc

In [5]:
def get_negihbors_val(neighbors_loc, img):
    neighbors_val = [img[y,x] for y, x in neighbors_loc]
    return neighbors_val

In [6]:
def get_negihbors_wt(coord, neighbors_loc): 
    ## 取的每個 neighbor 與 sample 點的距離
    neighbors_dist = LA.norm(coord - neighbors_loc, axis=1)
    
    ## 若與第一個 pixel 的距離是 0 (即在 pixel 上)則為當下的值
    ## 否則每個 neighbor 的權重為距離倒數
    neighbors_wt = neighbors_dist[0] == 0 and [1,0,0,0] or (1 - neighbors_dist)/3
    return neighbors_wt

In [7]:
def get_spl_val(coord, img):
    ## 取得有影響力的 4 個相鄰點及相對應的 值 和 權重
    neighbors_loc = get_influential_neighbors(coord)
    neighbors_val = get_negihbors_val(neighbors_loc, img)
    neighbors_wt  = get_negihbors_wt(coord, neighbors_loc)
    
    ## 計算 sample 的值
    value = np.dot(neighbors_val, neighbors_wt)
    return value
    

In [11]:
def get_sbp_edges(center, edge_pts, img):
    sbp_edges = []
    
    for pt in edge_pts:
        ## 取得中心到邊緣的單位向量
        c2e_V = pt - center
        c2e_V = c2e_V/LA.norm(c2e_V)
        
        spl_pts  = [pt + i*c2e_V for i in range(-10,11)]       # 需要再找道合適參數
        spl_vals = [get_spl_val(spl_pt, img) for spl_pt in spl_pts]
            
#         print(spl_pts)
#         print(spl_vals)
        
        x_label = np.asarray(range(0,21))

        ## 擬合 S 曲線
        popt, pcov = curve_fit(sigmoid, x_label, spl_vals)
#         print(popt)
        
        x = np.linspace(-5, 35, 50)
        y = sigmoid(x, *popt)

        plt.plot(x_label, spl_vals, 'o', label='data')
#         plt.plot(x,y, label='fit')
        plt.ylim(100, 140)
        plt.legend(loc='best')
        plt.show()
        
        break
    
    return sp_edges

In [12]:
# os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
os.chdir("/home/mj/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
file = "10_mean_80.bmp"
img = cv2.imread(file, 0)

time_start = time.time()

blur, nlm, otsu, edge = image_processing(img)
edge_points = get_edge_points(edge)
cx,cy,r,_ = cf.least_squares_circle((edge_points))
sp_edges = get_sbp_edges(np.asarray([cx,cy]), edge_points, nlm)


elapsed_time = time.time() - time_start
print(elapsed_time)
# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# print("{}, {}".format(cx, cy))

TypeError: iteration over a 0-d array